# Setup

In [3]:
# Imports
import os
import os.path as osp

import joblib
import numpy as np
import gym
import pandas as pd
import matplotlib.pyplot as plt

import pirl
from pirl import config, experiments

from analysis import common

In [5]:
# Config
experiments = {
    'easy': 'continuous-baselines-easy-20180603_030017-685c8b0a724b56fd7293c81e23efd993de935a73',
    'medium': 'continuous-baselines-medium-20180603_064854-685c8b0a724b56fd7293c81e23efd993de935a73',
}
experiment_dirs = {k: osp.join('..', 'data', 'aws_live', 'experiments', v) for k, v in experiments.items()}

# Value difference

In [10]:
algo_pattern = '(.*)'
env_pattern = '(.*)'
dfs = {k: common.load_value(v, algo_pattern, env_pattern)
       for k, v in experiment_dirs.items()}
df = pd.concat(list(dfs.values()), axis=0)
values = common.aggregate_value(df)

In [15]:
common.plot_ci(values)

airl so                  airl sa           airl random               ppo cts                expert
env                       n m    eval                                                                                                                    
HalfCheetah-v2            0 1000 irl      1475.063 +/- 222.229     2404.200 +/- 172.579  -4478.457 +/- 21.494  2292.573 +/- 167.836  2673.013 +/- 241.439
                                 ppo_cts  -260.050 +/- 142.574  -52271.410 +/- 3574.235   629.893 +/- 413.990  2292.573 +/- 167.836  2673.013 +/- 241.439
Hopper-v2                 0 1000 irl           2.903 +/- 0.145          4.183 +/- 0.343      12.860 +/- 2.392  2057.597 +/- 169.414  2057.597 +/- 169.414
                                 ppo_cts    348.417 +/- 91.602    -2476.667 +/- 593.255       3.170 +/- 0.133  2057.597 +/- 169.414  2057.597 +/- 169.414
InvertedDoublePendulum-v2 0 1000 irl          30.447 +/- 2.239       186.220 +/- 86.182      45.343 +/- 3.039  4970.857 +/- 665.367  5778.713 +/- 665.608
                                 ppo_cts  1085.687 +/- 470.688       197.427 +/- 24.981      27.300 +/- 0.497  4970.857 +/- 665.367  5778.713 +/- 665.608
InvertedPendulum-v2       0 1000 irl           3.003 +/- 0.011          3.437 +/- 0.100       4.597 +/- 0.514    722.500 +/- 73.104    882.893 +/- 53.569
                                 ppo_cts    578.547 +/- 91.625       610.273 +/- 81.633       2.680 +/- 0.092    722.500 +/- 73.104    882.893 +/- 53.569
Reacher-v2                0 1000 irl          -8.467 +/- 0.840         -6.650 +/- 0.594   -754.783 +/- 21.202      -6.277 +/- 0.575      -6.420 +/- 0.597
                                 ppo_cts     -70.877 +/- 6.070  -20751.833 +/- 2600.606     -12.350 +/- 1.279      -6.277 +/- 0.575      -6.420 +/- 0.597
Swimmer-v2                0 1000 irl          48.287 +/- 1.648         41.877 +/- 2.221      -1.180 +/- 2.506      61.383 +/- 5.801      46.840 +/- 1.984
                                 ppo_cts      26.377 +/- 1.807      -447.853 +/- 51.037       1.883 +/- 3.604      61.383 +/- 5.801      46.840 +/- 1.984

In [19]:
df.xs('ppo_cts', level='eval').xs('mean', level='type')

airl so   airl sa  airl random  ppo cts   expert
env                       n m    seed                                                  
Reacher-v2                0 1000 0      -70.58  -6905.48       -12.94    -7.81    -6.02
                                 1      -94.92 -38293.99       -12.99    -5.70    -6.88
                                 2      -47.13 -17056.03       -11.12    -5.32    -6.36
InvertedPendulum-v2       0 1000 0      608.18    780.43         2.04   982.70   866.41
                                 1      562.22    289.39         3.00   430.01   782.27
                                 2      565.24    761.00         3.00   754.79  1000.00
InvertedDoublePendulum-v2 0 1000 0       28.73    288.84        27.22  5837.12  6556.97
                                 1       99.06    230.41        27.64  5935.82  4894.22
                                 2     3129.27     73.03        27.04  3139.63  5884.95
Swimmer-v2                0 1000 0       20.95   -131.86       -12.90    99.90    56.27
                                 1       24.95   -670.90        -8.78    46.08    46.08
                                 2       33.23   -540.80        27.33    38.17    38.17
Hopper-v2                 0 1000 0      996.26    -48.76         4.07  2135.26  2135.26
                                 1        0.38   -807.06         2.48  2405.72  2405.72
                                 2       48.61  -6574.18         2.96  1631.81  1631.81
HalfCheetah-v2            0 1000 0    -1282.89 -32087.04      -709.55  2721.97  2721.97
                                 1      327.42 -75172.83      3559.85  1424.26  1424.26
                                 2      175.32 -49554.36      -960.62  2731.49  3872.81

# Policy rollout

In [ ]:
def expert_cached_value(rl, env_name, pol_discount=0.99, eval_discount=1.00, seed=1234, episodes=100):
    '''Rollout a cached expert policy for episodes.
       WARNING: This will be slow or just break if policy is not in cache!'''
    gen_policy, _sample, compute_value = config.RL_ALGORITHMS[rl]
    policy, value = experiments._train_policy(rl, pol_discount, env_name, seed, None)
    vmean, vse = value
    print('Cached value: {:.3f} +/- {:.3f}'.format(vmean, 1.96 * vse))
    
    env = gym.make(env_name)
    rmean, rse = compute_value(env, policy, eval_discount, num_episodes=episodes, seed=seed)
    print('Rollout value: {:.3f} +/- {:.3f}'.format(rmean, 1.96 * rse))
    return (vmean, vse), (rmean, rse)

def _policy_value(results_dir, rl, env_name, pol_discount, eval_discount, episodes, seed):
    _gen_policy, _sample, compute_value = config.RL_ALGORITHMS[rl]
    fname = osp.join(results_dir, 'policy.pkl')
    print('Loading policy from ', fname)
    policy = joblib.load(fname)
    env = gym.make(env_name)
    
    mean, se = compute_value(env, policy, eval_discount, num_episodes=episodes, seed=seed)
    print('Rollout value: {:.3f} +/- {:.3f}'.format(mean, 1.96 * se))
    return mean, se

def expert_value(experiment_dir, rl, env_name, pol_discount=0.99, eval_discount=1.00, episodes=100, seed=1234):
    results_dir = osp.join(experiment_dir, 'expert', env_name, rl)
    return _policy_value(results_dir, rl, env_name, pol_discount, eval_discount, episodes, seed)

def irl_eval_value(experiment_dir, irl_name, num_traj, rl, env_name, pol_discount=0.99, eval_discount=1.00, episodes=100, seed=1234):
    results_dir = osp.join(experiment_dir, 'eval', env_name, 
                           '{}:{}:{}'.format(irl_name, num_traj, num_traj), rl)
    return _policy_value(results_dir, rl, env_name, pol_discount, eval_discount, episodes, seed)
    
def irl_value(experiment_dir, irl_name, env_name, num_traj, eval_discount=1.00, episodes=100):
    _irl_algo, _reward_wrapper, compute_value = experiments.make_irl_algo(irl_name)
    irl_dir = osp.join(experiment_dir, 'irl', irl_name)
    if not os.path.exists(irl_dir):
        raise FileNotFoundError("No result directory {}".format(irl_dir))
    
    pop_fname = osp.join(irl_dir, str(num_traj), 'policies.pkl')
    sin_fname = osp.join(irl_dir, env_name, str(num_traj), 'policy.pkl')
    if os.path.exists(pop_fname):
        policies = joblib.load(pop_fname)
        print(policies.keys())
        policy = policies[env_name]
    elif os.path.exists(sin_fname):
        policy = joblib.load(sin_fname)
    else:
        raise FileNotFoundError("Neither {} or {} exists".format(pop_fname, sin_fname))
    
    env = gym.make(env_name)
    mean, se = compute_value(env, policy, discount=eval_discount, num_episodes=episodes)
    print('Rollout value: {} +/- {}'.format(mean, 1.96 * se))

In [ ]:
expert_cached_value('ppo_cts', 'Reacher-v2', episodes=100)

In [ ]:
expert_value(experiment_dir, 'ppo_cts', 'Reacher-v2', episodes=500, seed=1234)

In [ ]:
irl_value(experiment_dir, 'airl', 'Reacher-v2', 1000, episodes=100)

# Visualizing rewards (gridworld only)

In [ ]:
def show_heatmaps(irl_algo, kind='inline', out_dir=None, shape=(9,9), **kwargs):
    data = pd.read_pickle(osp.join(experiment_dir, 'results.pkl'))
    rewards = data['rewards'][irl_algo]
    if kind in ['inline', 'pdf']:
        figs = common.gridworld_heatmap(rewards, shape)
        if out_dir is None:
            for fig in figs:
                display(fig[1])
        else:
            common.save_figs(figs, out_dir)
    elif kind == 'movie':
        common.gridworld_heatmap_movie(out_dir, rewards, shape)
    else:
        assert False

In [ ]:
irl_algos = ['mce', 'mcep_reg1e0']
for irl in irl_algos:
    show_heatmaps(irl, kind='inline', shape=(4,4))
    #show_heatmaps(irl, kind='movie', out_dir='figs/jungle/movies/' + irl)
    #show_heatmaps(irl, kind='movie', out_dir='figs/jungle/' + irl)

# Loss curve (PPO only)

In [ ]:
def ppo_progress(results_dir):
    path = osp.join(results_dir, 'progress.csv')
    df = pd.read_csv(path)
    df = df.set_index('serial_timesteps')
    return df

def expert_ppo_progress(experiment_dir, env_name, rl_name):
    results_dir = osp.join(experiment_dir, 'expert', 
                       experiments.sanitize_env_name(env_name),
                       rl_name)
    return ppo_progress(results_dir)

In [ ]:
envs = ['InvertedPendulum-v2', 'InvertedDoublePendulum-v2', 'Reacher-v2']
for env in envs:
    df = expert_ppo_progress(experiment_dir, env, 'ppo_cts')
    plt.figure()
    df['eprewmean'].plot()
    print(df['eprewmean'].max())
    plt.title(env)

# Evaluate checkpoint

In [ ]:
def load_checkpoint(results_dir, checkpoint_num=None):
    checkpoint_dir = osp.join(results_dir, 'checkpoints')
    if checkpoint_num is None:
        checkpoint_num = max(os.listdir(checkpoint_dir))
    checkpoint_fname = osp.join(checkpoint_dir, checkpoint_num)
    print('Loading from ', checkpoint_fname)
    policy = joblib.load(checkpoint_fname)
    
    return policy

def ppo_value2(results_dir, rl, env_name, pol_discount, eval_discount, episodes):
    _gen_policy, _sample, compute_value = config.RL_ALGORITHMS[rl]
    fname = osp.join(results_dir, 'policy.pkl')
    print('Loading policy from ', fname)
    policy = joblib.load(fname)
    env = gym.make(env_name)
    
    mean, se = compute_value(env, policy, eval_discount, num_episodes=episodes)
    print('Rollout value: {:.3f} +/- {:.3f}'.format(mean, 1.96 * se))
    return mean, se

def ppo_value(policy, env_name, episodes, seed=1234):
    _, _, compute_value = config.RL_ALGORITHMS['ppo_cts']
    env = gym.make(env_name)
    mean, se = compute_value(env, policy, 1.00, num_episodes=episodes, seed=seed)
    print('Rollout value: {:.3f} +/- {:.3f}'.format(mean, 1.96 * se))

In [ ]:
policy = load_checkpoint(osp.join(experiment_dir, 'expert/InvertedPendulum-v2/ppo_cts'), '00488')
ppo_value(policy, 'InvertedPendulum-v2', 500, 1234)